## Create database

In [11]:
import sqlite3
from sqlite3 import Error

#https://www.sqlitetutorial.net/sqlite-python/

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

if __name__ == '__main__':
    #so cria o arquivo se a pasta sqlite e db existirem
    #se ao inves de passar o endereco do arquivo, eu passar :memory: o python cria um arquivo na memoria ram
    conn = create_connection(r"C:\sqlite\db\pythonsqlite.db")
    print(conn)

2.6.0
None


## Cria tabela estrela

In [13]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def main():
    database = "C:\sqlite\db\pythonsqlite.db"

    sql_create_historico_covid_brasil_table = """ CREATE TABLE IF NOT EXISTS historicoCovidBrasil (
                                        FOREIGN KEY (coduf_id) REFERENCES regiao(coduf)
                                        FOREIGN KEY(data_id) REFERENCES tempo(data)
                                        populacao integer,
                                    ); """

    sql_create_tempo_table = """ CREATE TABLE IF NOT EXISTS tempo (
                                        data text PRIMARY KEY,
                                        semanaEpi integer,
                                    );"""

    sql_create_populacao_table = """ CREATE TABLE IF NOT EXISTS populacao (
                                        populacaoTCU2019 integer,
                                        casosAcumulado integer,
                                        casosNovos integer,
                                        obitosAcumulado integer,
                                        obitosNovos integer
                                    );"""

    sql_regiao_table = """ CREATE TABLE IF NOT EXISTS regiao (
                                        coduf integer PRIMARY KEY,
                                        estado text,
                                        regiao text,
                                        codmun integer,
                                        codReg integer,
                                        nomeRegiaoSaude text,
                                    );"""

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create historico table
        create_table(conn, sql_create_historico_covid_brasil_table)
    else:
        print("Error! cannot create the database connection.")

In [ ]:
if __name__ == '__main__':
    main()